In [1]:
import pandas as pd
from gurobipy import *

#### 数据导入模块

In [2]:
f=r'data_v1.xlsx'
data_ferry=pd.read_excel(f,sheet_name="Sheet2-ferry")
data_port=pd.read_excel(f,sheet_name="Sheet3-1")

#### 参数设定

In [3]:
# time1-time2 这段时间不能出海
#早上5:00-6:00
time1=0
time2=6

In [4]:
m =3 #轮船数
n = 5 #港口数

q = 115 #时刻数
lam = 1 # 轮船成本系数
niu = 10 # 客户成本系数

F = range(0,m) #轮船集合
P = range(0,n) #港口集合
Q = range(0,q) #时刻集合

scalar=1

In [5]:
#参数list初始化
cap = [0] * m #轮船集合
home_port = [0] * m # 轮船的母港
berth = [0] * n #每个港口的可容纳的轮船数

In [6]:
w=[[0 for k in P] for f in F]   #w[ferry][port]=waiting time
i=0
j=0
for index,row in data_port.iterrows():
    port,time=row
    port=int(port[-1])
    j+=1
    w[i][port-1]=int(time/10)
    if j%5==0:
        i+=1

In [7]:
for i in F:
    cap[i] = data_ferry["Cf"][i]  #cap[f] capacity
    home_port[i] = data_ferry['hf'][i]-1  #home_port[f] home port

for i in P:
    berth[i] = 2

In [8]:
for i in F:
    cap[i]=cap[i]*scalar

In [9]:
data_ferry.head()

,f,hf,Cf,gf_enroute,gf_inport
0,small1,1,100.0,400.0,160.0
1,small2,3,100.0,400.0,160.0
2,large,5,200.0,600.0,240.0
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


#### 节点定义

In [10]:
data_demand=pd.read_excel(f,sheet_name="Sheet5-1")
# 列更名+删除无用行
data_demand.columns=['port','cus_port','series','volume','arrival_time']

In [11]:
# 根据sheet5生成所有node
V=[[] for i in P] # 对每个港口都生成node   V[port][time] 嵌套的list 
# V中node的格式：(初始port,时刻上的序号)
for i in P:
    for j in Q:        
        node=(i,j) 
        V[i].append(node)

In [12]:
VV=[]               #VV 所有node
for v in V:
    for i in v:
        VV.append(i)

In [13]:
U=VV[:]            #U 所有node并上 虚拟点
for i in P:
    U.append((i,q))

In [14]:
# 对demand列表循环，将客户需求、客户目的地、客户到达时间更新
D=[{} for t in P]                   #D[aim port][VV 中的有效的node]=demand
arr_time=[{} for t in P]            #arr_time[aim port][VV 中的有效的node]=一个具体的时刻
for t in P:
    for v in VV:
        D[t][v]=0
        arr_time[t][v]=q
for index,row in data_demand.iterrows():  
    i, a, j, b, c = row
    D[a-1][(i-1,j-1)]=b
    arr_time[a-1][(i-1,j-1)]=c-1

# 定义虚拟节点，其格式和其他节点一致
for i in P:
    # 先求在虚拟节点的客户净需求量
    virtual_demand=-data_demand[data_demand['cus_port']==(i+1)]['volume'].sum()
    D[i][(i,q)]=virtual_demand
    for t in P:
        if t!=i:
            D[i][(t,q)]=0

#### 弧定义 

In [15]:
def ceil(x):
    if int(x)==x:
        return int(x)
    else:
        return int(x)+1

In [16]:
# 先获取每条船可以往返的港口+单程时间
data_time=pd.read_excel(f,sheet_name='Sheet4-time')
# 排除不能到达的港口
data_time=data_time[data_time['tour_time']!=999]

##### 更换例子要改

In [17]:
# 定义轮船的arc
ferry_type=['small','small','large'] # 通过船的类型确定往返港口和单程时间
#ferry_type=['small','large'] # 通过船的类型确定往返港口和单程时间
E=[[] for f in F] # 定义每条轮船的edge集合                     E[ferry]=[]对于ferry f 的feasible arc
for f in F: # 对每条船都开始生成可行的弧
    temp_data=data_time[data_time['type']==ferry_type[f]] # 生成每种船对应的data
    for index,row in temp_data.iterrows():
        from_port, to_port, tour_time=row[1:]
        for i in Q:
            for j in range(i+1,q):                
                if from_port==to_port and j>i+1: # 生成同港口的弧
                    break
                try:
                    tour=max(1,ceil(float(tour_time/10)))
                except:
                    continue
                if i+tour!=j: # 从from_port(i)到to_port(j)是否可行的判断  
                    continue
                E[f].append((V[from_port-1][i],V[to_port-1][j]))

In [18]:
# 定义客户流的destination arc
Dest=[[] for k in P]                   #Dest[aim port]=相同港口的0-114的点 到相同港口115的 边
for k in P:
    for i in Q:
        Dest[k].append((V[k][i],(k,q)))

In [19]:
# 定义客户流的infeasible arc              
Infeas=[]                                #Infeas  所有这样交叉的 边  
Infeas_port=[[] for k in P]             #Infeas_port[aim port]=不通港口的114 到 目的港口115的 边
for k in P:
    for h in P:
        if k==h:
            continue
        Infeas.append((V[h][q-1],(k,q)))
        Infeas_port[k].append((V[h][q-1],(k,q)))

In [20]:
# 定义约束中需要的，符合一个图的边集
A=[[] for k in P]    #A[aim port] all service arc + infeas + dest 
EE=[]
for f in F:
    for a in E[f]:
        EE.append(a)   #EE 没有剔除重复arc的service arc
Euni=list(set(EE))   
for k in P:
    Atemp=[]
    Atemp=Euni+Infeas_port[k]+Dest[k]
    A[k]=list(set(Atemp))

#### 成本相关定义

#### 更换例子要改

In [21]:
fer_types=set(data_ferry['f'].iloc[:3])
cost={}
for fer_type in fer_types:
    temp_data=data_ferry[data_ferry['f']==fer_type]
    cost[fer_type]={'inport':float(temp_data['gf_inport']),'enroute':float(temp_data['gf_enroute'])}
    
# 定义每条船在每条弧上的成本
new_ferry_type=['small1','small2','large']
#new_ferry_type=['small1','large1']
g=[{} for f in F]      #g[ferry][arc] cost for ferry 
for f in F:
    for e in E[f]:
        node1, node2=e
        if node1[0]==node2[0]:
            g[f][e]=cost[new_ferry_type[f]]['inport']*(node2[1]-node1[1])/6
        else:
            g[f][e]=cost[new_ferry_type[f]]['enroute']*(node2[1]-node1[1])/6
            
# 定义客户流在每条弧上的成本
M=10e5 # 定义大M成本
C=[{} for k in P]
for k in P:
    for f in F:
        for e in E[f]:
            node1, node2=e
            C[k][e]=(node2[1]-node1[1])*10   #C[aim port][arc] cost for comsumers
        for h in Dest[k]:
            C[k][h]=0
        for e in Infeas:
            C[k][e]=M

#### 其他集合定义

In [22]:
I=[{} for f in F]             #I[f][VV node ]=[前序 node]  only for service arc
O=[{} for f in F]             #O[f][VV node ]=[后序 node]  only for service arc
for f in F:
    for v in VV:
        I[f][v]=[]
        O[f][v]=[]
        for t in E[f]:
            if t[1]==v:
                I[f][v].append(t[0])
            if t[0]==v:
                O[f][v].append(t[1]) 

In [23]:
b=[{}for f in F]                #b[f][VV node]=flow balance require
for f in F:
    for v in VV:
        hf=data_ferry['hf'][f]-1
        if v==(hf,0):
            b[f][v]=-1
        elif v==(hf,q-1):
            b[f][v]=1
        else:
            b[f][v]=0

In [24]:
Iserv=[{} for f in F]   #Iserv[f][VV node]=剔除了相同port 的 能真正运客的 前序点
for f in F:
    for v in VV:
        port=v[0]
        Iserv[f][v]=I[f][v][:]
        for t in Iserv[f][v]:
            if t[0]==port:
                Iserv[f][v].remove(t)

In [25]:
I_nof_serv={}                   #I_nof_serv[VV node]和ferry 无关的 和当前port不一样的 前序点
for v in VV:
    I_nof_serv[v]=[]
    for f in F:
        if v in Iserv[f].keys():
            for i in Iserv[f][v]:
                I_nof_serv[v].append(i)
            I_nof_serv[v]=list(set(I_nof_serv[v]))

In [26]:
delta=[{} for f in F]     #dealta[f][VV node]=向后的arc

for f in F:
    for v in VV:
        delta[f][v]=[]
        time=v[1]
        port=v[0]
        r=min(w[f][port],q-1-time)
        for i in range(1,r+1):
            if time+i<=q-1:
                delta[f][v].append(((port,time+i-1),(port,time+i)))

In [27]:
beta=[2,2,2,2,1]  #port's capacity 

In [28]:
d=[{}for t in P]        #demand d[aim port][VV node]
for t in P:
    for ki in VV:
        d[t][ki]={}
        for v in U:
            if v==ki:
                d[t][ki][v]=D[t][ki]
            elif v==(t,q):
                d[t][ki][v]=-D[t][ki]
            else:
                d[t][ki][v]=0

In [29]:
II=[{}for t in P]               #II[aim port][U node]  for客户流平衡方程
OO=[{}for t in P]               #OO[aim port][U node]  
for t in P:
    for v in U:
        II[t][v]=[] 
        OO[t][v]=[]
        for i in A[t]:
            if i[1]==v:
                '''if v[1]==115 and v[0]!=t:
                    continue'''
                II[t][v].append(i[0])
            if i[0]==v:
                if v[1]==q-1 and i[1][0]!=t:  #修改过2--114
                    continue
                OO[t][v].append(i[1])

In [30]:
FF={}            #FF[arc]  哪些ferry 可以走这条 arc
for arc in Euni:
    FF[arc]=[] # FF表示轮船图中包含arc的轮船
    for f in F:
        if arc in E[f]:
            FF[arc].append(f)        

### Model Definition

In [31]:
model=Model()
y={}
for f in F:
    for e in E[f]:
        y[f,e]=model.addVar(vtype=GRB.BINARY,name="y"+str(f)+str(e))

Academic license - for non-commercial use only - expires 2021-03-26
Using license file C:\Users\Zhangyt\gurobi.lic


In [32]:
x={}
for k in P:
    for t in P:
        if k==t:
            continue
        for i in Q:
            if i==q-1:
                continue
            for e in A[k]:
                x[V[t][i],k,e]=model.addVar(vtype=GRB.INTEGER,
                                            name="x-"+"t:"+str(t)+",i:"+str(i)+",k:"+str(k)+",e:"+str(e))  # V是出发点， k是目标port， e是feasible arc

In [33]:
z={}
for k in P:
    for t in P:
        if k==t:
            continue
        for i in Q:
            if i==q-1:
                continue
            for j in range(i+1,q):
                z[V[t][i],k,V[k][j]]=model.addVar(vtype=GRB.BINARY, name="z-"+"t"+str(t)+"i"+str(i)+"k"+str(k)+"j"+str(j))   # 第一个t,i 是出发点  k是目标port 第二个V是到达点

##### 目标函数
\begin{aligned}
\text{Minimize} \quad  & \lambda \sum_{f\in F} \sum_{e\in E^f} g_e^f y_e^f+{\color{red} { v \sum_{k \in P}\sum_{e\in A^{k}}c_e^{k} \sum_{v_i \in V,i\ne q }x_e^{v_i,k} }} \\ 
\end{aligned}

In [34]:
model.setObjective(lam*quicksum(g[f][e]*y[f,e] for f in F for e in E[f])+niu*quicksum(C[k][a]*x[V[t][i],k,a] for k in P for t in P if t!=k for i in Q 
                                                                                 if i!=q-1 for a in A[k]),GRB.MINIMIZE)

##### 1. 轮船流入流出约束
 \begin{aligned}
\sum_{v^{\prime} \in I^{f}(v)} y_{v^{\prime} v}^{f}-\sum_{v^{\prime} \in O^{f}(v)} y_{v v^{\prime}}^{f}=b_{v}^{f} \text { for every } v \in V \text { and } f \in F\\
\end{aligned}

In [35]:
model.addConstrs((quicksum(y[f,(v1,v)] 
                           for v1 in I[f][v])-
                  quicksum(y[f,(v,v2)] 
                           for v2 in O[f][v]) == b[f][v]
                  for v in VV for f in F),name="c1-"+"v"+str(v)+"f"+str(f))

{((0, 0), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 6), 

##### 2. 轮船必须停留一段时间的约束
\begin{aligned}
&\left|\Delta_{k_{i}}^{f}\right| \sum_{v^{\prime} \in I_{\text {serv }}^{f}\left(k_{i}\right)} y_{v^{\prime} k_{i}}^{f} \leq \sum_{(u, v) \in \Delta_{k_{i}}^{f}} y_{u v}^{f} \text { for every } k_{i} \in V \text { and } f \in F,\\
\end{aligned}
 $\Delta=2$

In [36]:
model.addConstrs((len(delta[f][ki]) * quicksum(y[f,(v1,ki)] for v1 in Iserv[f][ki]) <= quicksum(y[f,a] for a in delta[f][ki])
             for ki in VV for f in F), name="c2-"+"ki"+str(ki)+"f"+str(f)  )

{((0, 0), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 1), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 2), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 3), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 4), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 0): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 1): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 5), 2): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 6), 

##### 3. 同一个港口最多停留的轮船数约束
\begin{aligned}
\sum_{f \in F} y_{k_{i} k_{i+1}}^{f} \leq \beta_{k} \text { for every waiting arc }\left(k_{i}, k_{i+1}\right), k_{i} \in V_{k}, k \in P, i \neq q\\
\end{aligned}

In [37]:
model.addConstrs((quicksum(y[f,(V[k][i],V[k][i+1])] for f in F) 
                  <= beta[k] for i in range(0,q-1) for k in P), 
                name="c3-"+"k"+str(k))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


##### 4. 客户的流入流出约束
\begin{aligned}
\sum_{v^{\prime} \in I^{k_i,t}_{v
 } } x_{v^{\prime}, v}^{k_i,t}-\sum_{v^{\prime} \in O_{v}^{k_i,t}} x_{v, v^{\prime}}^{k_i,t}=-d_{v}^{k_i,t} \text { for every v } \in U^{k_i,t} \text { and for every } (k_i,t) \in (V\times P)\\
\end{aligned}
$I^{k_i,t}_{v }$ 用$I^{t}_{v }$就可以了，同理$O$,$U$

In [38]:
model.addConstrs((quicksum(x[a,t,(u1,v)] for u1 in II[t][v])-
                  quicksum(x[a,t,(v,u2)] for u2 in OO[t][v]) ==
                  -d[t][a][v] for v in U for t in P
            for a in VV if a[0]!=t if a[1]!=q-1),name="c4-"+"v"+str(v)+"t"+str(t)+"a"+str(a))

{((0, 0), 0, (1, 0)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 1)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 2)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 3)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 4)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 5)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 6)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 7)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 8)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 9)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 10)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 11)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 12)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 13)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 14)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 0), 0, (1, 15)): <gurobi.Constr *Awaiting Mo

##### 5. 客户流量<=轮船容量约束
\begin{aligned}
 \sum_{(k_i,t) \in V\times P} x_{u v}^{k_i,t} \leq \sum_{f \in F(u, v)} C^{f} y_{u v}^{f} \text { for every service } \operatorname{arc}(u, v) \in \Omega\\
 \end{aligned}

In [39]:
model.addConstrs((quicksum(x[a,t,(uu,v)] for a in VV if (a[1]<=uu[1] and a[1]<=v[1]) for t in P if a[0]!=t) <= 
            quicksum(y[f,(uu,v)]*cap[f] for f in FF[(uu,v)]) for (uu,v) in Euni if uu[0] != v[0] ) , name="c5-" )

{((1, 5), (4, 12)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 72), (4, 79)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 69), (0, 73)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 4), (4, 11)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 26), (4, 33)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 70), (1, 73)): <gurobi.Constr *Awaiting Model Update*>,
 ((0, 35), (3, 39)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 6), (3, 9)): <gurobi.Constr *Awaiting Model Update*>,
 ((4, 106), (1, 113)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 40), (1, 43)): <gurobi.Constr *Awaiting Model Update*>,
 ((2, 43), (0, 48)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 68), (4, 75)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 61), (4, 68)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 81), (2, 86)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 102), (3, 105)): <gurobi.Constr *Awaiting Model Update*>,
 ((1, 25), (4, 32)): <gurobi.Constr *Awaiting Model Update*>,
 ((3, 76

##### 6. 客户转移时间
 \begin{aligned}
{
  \sum_{j=i}^{t} \sum_{v^{\prime} \in I_{\text {serv }}\left(k_{j}\right)} x_{v^{\prime} k_{j}}^{k_i,p} \leq x_{k_{t} k_{t+1}}^{k_i,p} \text { for } t=i \text { to } r-1 ,k_{i} \in V, i \neq q \text { and } p \in P \backslash\{k\} }
\end{aligned}

 只停留一期
\begin{aligned}
{ \sum_{v^{\prime} \in I_{\text {serv }}\left(k_{i}\right)} x_{v^{\prime} k_{i}}^{k_i,p} \leq x_{k_{i} k_{i+1}}^{k_i,p} , k_{i} \in V, i \neq q \text { and } p \in P \backslash\{k\} }
 \end{aligned}

In [40]:
model.addConstrs((quicksum(x[V[k][i],pp,(v,V[k][i])] for v in I_nof_serv[(k,i)]) <= 
            x[V[k][i],pp,(V[k][i],V[k][i+1])]  for k in P for i in Q if i!=q-1 for pp in P if pp!=k),
                name="c6-")

{(0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 3): <gurobi.Constr *Awaiting Model Upd

 ##### 7. 硬时间窗
$$\sum_{v^{\prime}\in I_{serve}(p_j)}x_{v^{\prime},p_j}^{k_i,p}\le M z_{p_j}^{k_i,p}  \qquad (1)$$

$$\tau(p_j)\le \tau (a(k_i,p))+M(1-z_{p_j}^{k_i,p}  )\qquad (2)$$

等价于 $$ j\le a(k_i,p)+M(1-z_{p_j}^{k_i,p}  )\qquad (2)$$

 $$(1),(2) \quad \forall k_i\in V,i\ne q,k\ne p,\forall p\in P, {\color{red}{ \forall j\in (i,q] }}$$

In [41]:
M=10**8

'M=10**8'

In [42]:
model.addConstrs( quicksum(x[V[k][i],p,(v,V[p][j])] 
                           for v in I_nof_serv[(p,j)]) 
                 <= M*z[V[k][i],p,V[p][j]] 
            for k in P for i in Q if i!=q-1 for p in P if p!=k for j in range(i+1,q))

'model.addConstrs( quicksum(x[V[k][i],p,(v,V[p][j])] \n                           for v in I_nof_serv[(p,j)]) \n                 <= M*z[V[k][i],p,V[p][j]] \n            for k in P for i in Q if i!=q-1 for p in P if p!=k for j in range(i+1,q))'

In [43]:
model.addConstrs(j <= arr_time[p][(k,i)]+M*(1-z[V[k][i],p,V[p][j]]) for k in P for i in Q if i!=q-1 for p in P if p!=k for j in range(i+1, q))

'model.addConstrs(j <= arr_time[p][(k,i)]+M*(1-z[V[k][i],p,V[p][j]]) for k in P for i in Q if i!=q-1 for p in P if p!=k for j in range(i+1, q))'

##### 8.新约束
$ \sum_{(u,v)\in E^f } y^f_{(u,v)} $ for every service arc during $[time1,time2]$ for every $f$

In [44]:
model.addConstrs(y[f,(u,v)]== 0 for f in F for (u,v) in E[f] if (u[0]!=v[0] and u[1] in  range(time1,time2)))

'model.addConstrs(y[f,(u,v)]== 0 for f in F for (u,v) in E[f] if (u[0]!=v[0] and u[1] in  range(time1,time2)))'

In [45]:
model.Params.timelimit = 10*60*60

Changed value of parameter timelimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [46]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1330676 rows, 6213646 columns and 14391964 nonzeros
Model fingerprint: 0x49be8da9
Variable types: 0 continuous, 6213646 integer (137446 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+01, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Presolve removed 159077 rows and 1093955 columns (presolve time = 5s) ...
Presolve removed 590803 rows and 3050906 columns (presolve time = 10s) ...
Presolve removed 729765 rows and 3710000 columns (presolve time = 16s) ...
Presolve removed 1172283 rows and 5705740 columns (presolve time = 20s) ...
Presolve removed 1315512 rows and 6159134 columns (presolve time = 26s) ...
Presolve removed 1315607 rows and 6159258 columns
Presolve time: 26.22s
Presolved: 15069 rows, 54388 columns, 157373 nonzeros
Variable types: 0 continuous, 54388 integ

     0     0 1532523.64    0 2548 6.4216e+08 1532523.64   100%     -  164s
     0     0 1533223.83    0 2552 6.4216e+08 1533223.83   100%     -  167s
     0     0 1533414.18    0 2578 6.4216e+08 1533414.18   100%     -  168s
     0     0 1533491.80    0 2554 6.4216e+08 1533491.80   100%     -  169s
     0     0 1534966.45    0 2608 6.4216e+08 1534966.45   100%     -  173s
     0     0 1535269.13    0 2429 6.4216e+08 1535269.13   100%     -  176s
     0     0 1535329.31    0 2480 6.4216e+08 1535329.31   100%     -  176s
     0     0 1536161.53    0 2551 6.4216e+08 1536161.53   100%     -  180s
     0     0 1536416.76    0 2400 6.4216e+08 1536416.76   100%     -  182s
     0     0 1536445.41    0 2455 6.4216e+08 1536445.41   100%     -  183s
H    0     0                    6.420927e+08 1536445.41   100%     -  186s
     0     0 1537190.38    0 2439 6.4209e+08 1537190.38   100%     -  186s
     0     0 1537190.38    0 2423 6.4209e+08 1537190.38   100%     -  191s
H    0     0             

  1784   857 1701234.10  124 3780 1912940.00 1610880.18  15.8%   787 1485s
  1789   861 1714988.82   97 3867 1912940.00 1610982.48  15.8%   785 1492s
  1790   861 1781477.29  134 3815 1912940.00 1611158.05  15.8%   785 1504s
  1791   862 1708577.14   98 3794 1912940.00 1611187.29  15.8%   784 1505s
  1796   865 1679493.75   84 3941 1912940.00 1611226.03  15.8%   782 1513s
H 1796   819                    1899140.0000 1611226.03  15.2%   782 1521s
H 1796   776                    1892720.0000 1611226.03  14.9%   782 1521s
  1797   777 1611634.80   40 4005 1892720.00 1611634.80  14.9%   782 1527s
  1799   778 1611809.89   11 3919 1892720.00 1611809.89  14.8%   781 1530s
H 1802   739                    1871120.0000 1611853.92  13.9%   779 1537s
  1804   741 1682954.79  119 3935 1871120.00 1612007.17  13.8%   779 1542s
  1806   742 1612074.78   19 3903 1871120.00 1612074.78  13.8%   778 1545s
  1810   745 1612146.52   22 3900 1871120.00 1612146.52  13.8%   776 1555s
  1815   748 1701234.10  

  2044   862 1785694.12  151 3856 1850660.00 1622413.44  12.3%   788 2164s
  2045   863 1622415.63   22 3869 1850660.00 1622415.63  12.3%   787 2165s
  2048   865 1622437.72   52 3986 1850660.00 1622437.72  12.3%   786 2171s
  2050   866 1622447.26   46 3934 1850660.00 1622447.26  12.3%   785 2175s
  2056   871 1622463.31   16  584 1850660.00 1622463.31  12.3%   860 2180s
  2057   872 1622463.31   34 1223 1850660.00 1622463.31  12.3%   860 2186s
  2058   873 1697052.09   87 1940 1850660.00 1622463.31  12.3%   860 2194s
  2059   873 1622463.31   19 2544 1850660.00 1622463.31  12.3%   859 2203s
  2060   874 1670345.20   57 2564 1850660.00 1622463.31  12.3%   859 2214s
  2061   875 1622463.31   13 2837 1850660.00 1622463.31  12.3%   858 2226s
  2062   875 1654729.04  104 3028 1850660.00 1622463.31  12.3%   858 2236s
  2063   876 1760417.11  128 3134 1850660.00 1622463.31  12.3%   858 2246s
  2064   877 1622463.31   11 3411 1850660.00 1622463.31  12.3%   857 2256s
  2065   877 1622463.31  

  2287  1027 1701234.10  124 4279 1850660.00 1634883.44  11.7%   855 3026s
  2290  1029 1781477.29  134 4283 1850660.00 1634904.42  11.7%   854 3031s
  2291  1029 1708577.14   98 4224 1850660.00 1635002.97  11.7%   854 3040s
  2296  1033 1679493.75   84 4343 1850660.00 1635037.70  11.7%   852 3046s
  2297  1033 1635127.14   40 4368 1850660.00 1635127.14  11.6%   852 3056s
  2301  1036 1664795.80   70 4327 1850660.00 1635149.62  11.6%   850 3062s
  2302  1037 1635159.15   26 4265 1850660.00 1635159.15  11.6%   850 3071s
  2304  1038 1682954.79  119 4308 1850660.00 1635162.50  11.6%   849 3075s
  2305  1039 1635167.47   60 4308 1850660.00 1635167.47  11.6%   849 3084s
  2306  1039 1635172.19   19 4328 1850660.00 1635172.19  11.6%   848 3085s
  2308  1041 1635177.21  101 4334 1850660.00 1635177.21  11.6%   847 3090s
  2309  1041 1786102.38  136 4327 1850660.00 1635185.42  11.6%   847 3099s
  2310  1042 1635187.57   22 4317 1850660.00 1635187.57  11.6%   847 3103s
  2311  1043 1635220.28  

  8838  4772 1736733.62   61 3295 1832900.00 1670301.29  8.87%  1548 33722s
H 8856  4706                    1831500.0000 1670301.29  8.80%  1550 33722s
H 9135  3800                    1801380.0000 1670301.29  7.28%  1535 33937s
  9501  3885 1718920.55   63 2331 1801380.00 1672761.17  7.14%  1521 34096s
  9784  4095 1682437.14   94 2994 1801380.00 1673405.98  7.10%  1522 34287s
 10065  4252 1713575.08   81 1654 1801380.00 1673783.86  7.08%  1524 34502s
 10299  4565 1715851.30   79 1970 1801380.00 1675476.91  6.99%  1532 34791s
 10708  4837 1747862.48  101 2715 1801380.00 1675542.97  6.99%  1531 35037s
 11127  5072 1693632.05   71 1704 1801380.00 1676545.76  6.93%  1522 35267s
 11417  5358 1683763.26   68 2030 1801380.00 1676931.37  6.91%  1523 35530s
 11855  5504 1688616.29   63 3400 1801380.00 1677201.39  6.89%  1514 35778s
 12148  5679 1718611.57   82 2280 1801380.00 1678273.18  6.83%  1517 36000s

Cutting planes:
  Gomory: 3
  Cover: 2
  Implied bound: 177
  Projected implied bound: 

In [47]:
print(model.objVal)

1801380.0000000002


### 输出

1. y:每艘船从自己的起点开始，然后获取后序节点，如果为1，放入list中。 $O[f][v]$
2. x: (ki,p)就能够识别一组客户流，所以只要一直追踪就可以了。需要在所有船，所有后序节点中寻找$O$。

In [48]:
route=[[]for f in F]
for f in F:
    #home_port
    start=(home_port[f],0)
    route[f].append(start)
    for i in range(q-1):   
        pre=route[f][i]
        no_stop=0  
        for v in O[f][pre]:
            if y[f,(pre,v)].x==1:
                route[f].append(v)
                no_stop=1
        if no_stop==0:
            break

In [49]:
res_y=[route[0],route[1],route[2]]
df=pd.DataFrame(res_y)  

In [50]:
f2=r'res_y_model1.xlsx'
df.to_excel(f2)

In [52]:
#出发点k,出发时间i,去往p，经过的arc e,数量d
D=range(42)
res_x=[{} for i in D]
for index,row in data_demand.iterrows():
    k,t,i,b,a = row
    k=k-1
    t=t-1
    i=i-1
    a=a-1
    start=(k,i)
    nodes=[start]
    while len(nodes)!=0:
        pre_nodes=nodes[:]
        nodes=[]
        for pre in pre_nodes:
            for v in OO[t][pre]:
                if x[V[k][i],t,(pre,v)].x !=0:
                    res_x[index-1][(pre,v)]=x[V[k][i],t,(pre,v)].x
                    nodes.append(v)

In [53]:
result=[]
for k in res_x[0].keys():
    res={}
    res['arc'+str(0)]=k
    res['quantity'+str(0)]=res_x[0][k]
    result.append(res)
data_x=pd.DataFrame(result)

In [54]:
for d in range(1,42):
    result=[]
    for k in res_x[d].keys():
        res={}
        res['arc'+str(d)]=k
        res['quantity'+str(d)]=res_x[d][k]
        result.append(res)
    data=pd.DataFrame(result)    
    data_x=data_x.join(data)

In [55]:
f3=r'res_x_model1.xlsx'
data_x.to_excel(f3)